If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers transformers[torch] accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import csv
import requests
import pandas as pd

import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import evaluate

import transformers

from datasets import Dataset

print(transformers.__version__)
print(sklearn.__version__)

4.35.2
1.2.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/text-classification).

# Fine-tuning a model on a text classification task

Some initial parameters for starting our model, even if it's not ideal for our
task of PNEUMONIA classification

In [5]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

In [6]:
# put a dl=1 here
#CSV_URL = 'https://www.dropbox.com/scl/fi/4x8aj95l7e9x96f4qzch1/mimic2_pneumonia_corpus.csv?rlkey=9rgtu2cp7wfv4rbpx3kw36a7g&dl=0'
CSV_URL = 'https://www.dropbox.com/scl/fi/4x8aj95l7e9x96f4qzch1/mimic2_pneumonia_corpus.csv?rlkey=9rgtu2cp7wfv4rbpx3kw36a7g&dl=1'

df = pd.read_csv(CSV_URL)

print(df.head())

   Unnamed: 0  subject_id  hadm_id             admit_dt  Pneumonia  \
0           5          37    18052  3264-08-14 00:00:00          1   
1          14          94     8743  2656-08-18 00:00:00          1   
2          10         117    14296  3131-11-27 00:00:00          1   
3          19         184      203  3251-04-30 00:00:00          1   
4          18         184    17249  3251-03-19 00:00:00          1   

                                                text  
0  \n\n\n     DATE: [**3264-8-14**] 10:57 AM\n   ...  
1  \n\n\n     DATE: [**2656-8-19**] 4:17 PM\n    ...  
2  \n\n\n     DATE: [**3131-11-28**] 1:30 PM\n   ...  
3  \n\n\n     DATE: [**3251-5-1**] 3:18 PM\n     ...  
4  \n\n\n     DATE: [**3251-3-19**] 3:18 PM\n    ...  


In [7]:
# now that we have a dataframe, here's a way to iterate through the rows

all_dataset_dicts = []

for index, row in df.iterrows():
  text = row['text']
  label = row['Pneumonia']

  # key values of text and label
  row_dict = {'text': text, 'label': label}
  all_dataset_dicts.append(row_dict)

print(f'len(all_dataset_dicts): {len(all_dataset_dicts)}')

len(all_dataset_dicts): 200


In [8]:
# now that we have all of the data, let's turn this into a type (Dataset) which HuggingFace recognizes

dataset_before_split = Dataset.from_list(all_dataset_dicts)

In [9]:
# now let's split this up into train and test:

dataset = dataset_before_split.train_test_split(test_size=0.3)

print(type(dataset))

<class 'datasets.dataset_dict.DatasetDict'>


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 140
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 60
    })
})

To access an actual element, you need to select a split first, then give an index:

In [11]:
dataset["train"][0]

{'text': '\n\n\n     DATE: [**2644-1-17**] 6:37 AM\n     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 12564**]\n     Reason: NGT placement                                               \n     Admitting Diagnosis: HEAD BLEED\n     ______________________________________________________________________________\n     UNDERLYING MEDICAL CONDITION:\n      [**Age over 90 **] year old woman with SAH                                                      \n     REASON FOR THIS EXAMINATION:\n      NGT placement                                                                   \n     ______________________________________________________________________________\n                                     FINAL REPORT\n     INDICATION:  A [**Age over 90 **]-year-old woman with SAH.  Nasogastric tube placement.\n     \n     TECHNIQUE:  Portable AP chest radiograph.\n     \n     There is no previous chest radiograph for comparison.\n     \n     FINDINGS:\

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [12]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(dataset["train"])

,text,label
0,"\n\n\n DATE: [**2674-11-21**] 6:23 AM\n CHEST (PORTABLE AP) Clip # [**Clip Number (Radiology) 2255**]\n Reason: 88 yo woman with milf renal failure, urtacaria from allergic\n ______________________________________________________________________________\n UNDERLYING MEDICAL CONDITION:\n 88 year old woman with \n REASON FOR THIS EXAMINATION:\n 88 yo woman with milf renal failure, urtacaria from allergic reaction now with\n wheezing after IVF. rule out failure.\n ______________________________________________________________________________\n FINAL REPORT\n INDICATION: Wheezing after IV hydration. Rule-out CHF.\n \n COMPARSIONS: [**2674-11-20**].\n \n PORTABLE AP CHEST: Cardiac, mediastinal and hilar contours are stable. The\n pulmonary vascularity is normal. There is persistent collapse or\n consolidation of the left lower lobe.\n \n IMPRESSION: No CHF. No significant change from study done 6.5 hours prior.\n\n",1
1,"\n\n\n DATE: [**2915-8-10**] 12:01 PM\n CHEST (PORTABLE AP) Clip # [**Clip Number (Radiology) 3702**]\n Reason: after bronchoscopy and intubation. improvement?\n ______________________________________________________________________________\n UNDERLYING MEDICAL CONDITION:\n Patient w/ aspiration pneumonia, extubated yesterday, continued large\n secretions.\n REASON FOR THIS EXAMINATION:\n after bronchoscopy and intubation. improvement?\n ______________________________________________________________________________\n FINAL REPORT\n HISTORY: Aspiration pneumonia, extubated yesterday with continued secretions.\n Following bronchoscopy and intubation. ? improvement.\n \n CHEST, SINGLE AP PORTABLE VIEW:\n \n Compared with one day earlier, an NG tube has been placed. The tip lies\n approximatley 2.1 cm above the carina, pointing slightly towards the right.\n There has been considerable interval improvement in the appearance of the\n right lung, with complete reexpansion of the right lung. The mediastinum is\n appropriately positioned. No pleural effusion or focal consolidation is seen.\n There is slight prominence of the vascular structures in the right lung. At\n the base, this may relate to atelectasis, but the possibility of some\n pulmonary plethora due to reexpansion should also be considered.\n \n IMPRESSION:\n 1. Considerable interval reexpansion of the right lung, now almost completely\n normal.\n 2. Pulmonary vascular plethora, which could relate to reexpansion physiology.\n 3. Subsegmental right lower lobe atelectasis.\n\n",1
2,"\n\n\n DATE: [**2919-8-31**] 5:14 PM\n CHEST (PA & LAT) Clip # [**Clip Number (Radiology) 11222**]\n Reason: infiltrate vs. pulmonary edema\n ______________________________________________________________________________\n UNDERLYING MEDICAL CONDITION:\n 50 year old man with cough and hemoptysis and now progressive O2 requirement,\n signs of CHF on exam. PMH of MVR.\n REASON FOR THIS EXAMINATION:\n infiltrate vs. pulmonary edema\n ______________________________________________________________________________\n FINAL REPORT\n INDICATIONS: Cough and hemoptysis.\n \n PA AND LATERAL CHEST: Comparison is made to previous films from [**2919-8-17**].\n The patient has prior MBR and median sternotomy.\n \n Note is made of increased nodular and linear opacities involving the right\n upper lobe and left lower lobe. In this patient with HIV and history of\n chemotherapy for seminoma, these findings are worrisome for chronic infection\n including fungus or microbacterial infection. Further evaluation with CT scan\n is recommended. There is slight prominence of the left hilum. The patchy\n opacity in the left lower lobe could represent bronchopneumonia. The heart\n is normal in size. There is no evidence for CHF. No pneumothorax is\n identified. \n \n IMPRESSION: Increased nausea and linear opacities involving the right upper\n lobe and left lower lobe, please see comment for discussion.\n\n",1
3,"\n\n\n DATE: [**2717-5-23**] 12:06 PM\n CHEST (PORTABLE AP); -76 BY SAME PHYSICIAN [**N

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [15]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Fine-tuning the model

Now that our data is ready, we can download a pretrained base model and fine-tune it. Since all our task is document classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

In [16]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [17]:
metric = evaluate.load("accuracy")

In [18]:
metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-pneumonia",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

Before we continue, we need to tokenize (translate into input_ids for training a model)

In [19]:
max_tokens = 512

In [20]:

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=max_tokens, add_special_tokens = True)


In [21]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/140 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/60 [00:00<?, ? examples/s]

In [22]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [25]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.563735,0.766667,0.587778,0.766667,0.665409
2,No log,0.540490,0.766667,0.587778,0.766667,0.665409
3,No log,0.582269,0.766667,0.587778,0.766667,0.665409
4,No log,0.545484,0.766667,0.587778,0.766667,0.665409
5,No log,0.538278,0.766667,0.587778,0.766667,0.665409


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=45, training_loss=0.6320330301920573, metrics={'train_runtime': 59.7967, 'train_samples_per_second': 11.706, 'train_steps_per_second': 0.753, 'total_flos': 92727179059200.0, 'train_loss': 0.6320330301920573, 'epoch': 5.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [26]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.5637350082397461,
 'eval_accuracy': 0.7666666666666667,
 'eval_precision': 0.5877777777777777,
 'eval_recall': 0.7666666666666667,
 'eval_f1': 0.6654088050314467,
 'eval_runtime': 1.1733,
 'eval_samples_per_second': 51.139,
 'eval_steps_per_second': 3.409,
 'epoch': 5.0}